In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from plotnine import *

In [7]:
class hybrid_recommendation_model():
    def __init__(self):
        self.load_data()


    def load_data(self):
        self.STEAM_USERS  = pd.read_csv("data/process_data/steam_users.csv", encoding="UTF-8")
        self.STEAM_GAMES = pd.read_csv("data/process_data/steam_games.csv", encoding="UTF-8")

    def split_data(self):
        train_users_data, test_users_data = train_test_split(self.STEAM_USERS, test_size= 0.2, random_state=42)
        train_users_exists =  train_users_data['user'].unique()
        # Lọc các user trong tập test không có trong tập train 
        train_non_exists = test_users_data[~test_users_data['user'].isin(train_users_exists)]

        # thêm các user trong tập test không có trong tập train
        self.train_users_data = pd.concat([train_users_data, train_non_exists])
        # Lọc các user tập test có trong tập train 
        self.test_users_data = test_users_data[test_users_data['user'].isin(train_users_exists)]

    def collaborative_filtering_model(self):
        # Tạo ma trận user-game
        self.utility_matrix = self.train_users_data.pivot_table(index='name', columns='user', values='hours_rating', fill_value=0)

        # Tính độ tương đồng game-game
        self.game_similarity_matrix = cosine_similarity(self.utility_matrix)

    def content_based_filtering_model(self):
        # Tạo một vectorizer TF-IDF cho các đặc trưng nội dung
        tfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

        # Tạo ma trận TF-IDF từ các đặc trưng nội dung (ví dụ: name, tag, details,description)
        tfidf_matrix = tfidf_vectorizer.fit_transform(self.STEAM_GAMES['tags']+ ' ' + self.STEAM_GAMES['details'] + ' ' + self.STEAM_GAMES['description'])

        # Thiết lập kích thước chunk cho việc xử lý ma trận TF-IDF
        chunk_size = 800

        # Lấy số lượng chunk
        num_chunks = tfidf_matrix.shape[0] // chunk_size + 1

        # Khởi tạo ma trận tương tự cosine
        self.cosine_similarities = np.zeros((tfidf_matrix.shape[0], tfidf_matrix.shape[0]), dtype=np.float32)

        # Tính toán ma trận tương tự cosine theo từng phần
        for i in range(num_chunks):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, tfidf_matrix.shape[0])

            # Lấy đoạn ma trận TF-IDF
            chunk_matrix = tfidf_matrix[start_idx:end_idx]


            # Tính độ tương tự cho từng đoạn
            chunk_cosine_similarities = cosine_similarity(chunk_matrix, tfidf_matrix)

            # Cập nhật ma trận độ tương đồng cosine bằng các giá trị của đoạn con
            self.cosine_similarities[start_idx:end_idx, :] = chunk_cosine_similarities

    def fit(self):
        self.split_data()
        self.collaborative_filtering_model()
        self.content_based_filtering_model()

    def compute_prediction(self, userid, itemid):
        user_rating = self.utility_matrix.loc[:,userid]
        index = self.utility_matrix.index.get_loc(itemid)
        item_similarity = self.game_similarity_matrix[index]
        
        numerate = np.dot(user_rating, item_similarity)
        denom = item_similarity[user_rating > 0].sum()
                
        if denom == 0 or numerate == 0:
            return user_rating[user_rating>0].mean()
        
        return numerate / denom

    def compute_rmse(self, **kwargs):
        test_set = self.test_users_data[['user', 'name']].to_numpy()
        test_real = self.test_users_data['hours_rating'].to_numpy()
        pred = []
        for data in test_set:
            res = self.compute_prediction(userid = data[0],
                            itemid = data[1],
                            **kwargs)
            pred.append(res)

        rmse = np.sqrt(np.mean(test_real - pred)**2)
        return rmse

    def calculate_user_rating(self, user_id, similarity_mtx, utility):
        user_rating = utility.loc[:,user_id]
        pred_rating = user_rating.copy()

        default_rating = user_rating[user_rating>0].mean()
        numerate = np.dot(similarity_mtx, user_rating)
        corr_sim = similarity_mtx[:, user_rating >0]
        
        for i, rating in enumerate(pred_rating):
            temp = 0
            if rating < 1:
                w_r = numerate[i]
                sum_w = corr_sim[i,:].sum()
                if w_r == 0 or sum_w == 0:
                    temp = default_rating
                else:
                    temp = w_r / sum_w
                pred_rating.iloc[i] = temp
        return pred_rating

    def recommendation(self, user, game, top_n):
        n = round(top_n/2)

        ### show Content-based filtering
        game_index = self.STEAM_GAMES[self.STEAM_GAMES['name'] == game].index[0]
        # Lấy danh sách các trò chơi được gợi ý có độ tương đồng giảm dần
        similar_indices = sorted(enumerate(self.cosine_similarities[game_index]), key=lambda x: x[1], reverse=True)
        game_data = self.STEAM_GAMES.iloc[game_index]
        
        ### show Collaborative filtering
        user_rating = self.utility_matrix.loc[:,user]
        pred_rating = self.calculate_user_rating(user, self.game_similarity_matrix, self.utility_matrix)

        top_item = sorted(range(1,len(pred_rating)), key = lambda i: -1*pred_rating.iloc[i])
        top_item = list(filter(lambda x: user_rating.iloc[x]==0.0, top_item))[:top_n-n]
        
        # Result
        print(f"Những game có thể [{user}] yêu thích:")
        for i in top_item:
            print(f"\tName: {self.utility_matrix.index[i]} \n\tRating: {pred_rating.iloc[i]}\n")


        print(f"Những game tương đồng với [{game_data['name']}] thể loại: \n[{game_data['tags']}]")
        recommended_games_data = self.STEAM_GAMES.iloc[[i[0] for i in similar_indices[1:n+1]]]
        for _, data in recommended_games_data.iterrows():
            print(f"\tName: {data['name']} \n\tTags: {data['tags']}\n")

        

In [8]:
model = hybrid_recommendation_model()
model.fit()

In [9]:
rmse = model.compute_rmse()
print(f"RMSE = {rmse}")

RMSE = 0.06849331593011844


In [11]:
user = 76767
game = "Dota 2"
model.recommendation(user, game, top_n=10)

Những game có thể 76767 yêu thích:
Name: Medieval II Total War 
Rating: 4.3267992547731975

Name: Counter-Strike Condition Zero 
Rating: 4.315199756024984

Name: Call of Duty Black Ops 
Rating: 4.239297692913273

Name: Half-Life 2 Deathmatch 
Rating: 4.187592532338756

Name: Battlefield Bad Company 2 
Rating: 4.183050907947678

Những game tương đồng với Dota 2 thể loại: 
[Free to Play,MOBA,Multiplayer,Strategy,e-sports,Team-Based,Competitive,Action,Online Co-Op,PvP,Difficult,Co-op,RTS,Tower Defense,Fantasy,RPG,Character Customization,Replay Value,Action RPG,Simulation]
Name: The International 2019 Battle Pass 
Tags: Action,Strategy,Free to Play

Name: The International 2019 Battle Pass - Level 50 
Tags: Action,Strategy,Free to Play

Name: The International 2019 Battle Pass - Level 100 
Tags: Free to Play,Strategy,Action

Name: 王者英雄 The Great Hero 
Tags: Early Access,Early Access,Strategy,RPG,Action,MOBA,Multiplayer,Indie,Singleplayer

Name: Prime World 
Tags: Free to Play,MOBA,Strategy